In [1]:
from datetime import datetime
import pandas as pd
import pandas_datareader.data as web
from sklearn.preprocessing import scale
# replaces pyfinance.ols.PandasRollingOLS (no longer maintained)
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

pd.set_option("mode.copy_on_write", True)


DATA_STORE = './data/assets.h5'
START = 2005
END = 2024

In [2]:
def download_data(ticker_list):
    data = yf.download(
        
        # passes the ticker
        tickers=ticker_list,
        
        # used for access data[ticker]
        group_by='ticker',
    
    )
    return data

def process_data(data):

    data = data.stack(level=0, future_stack=True)
    data = data.dropna()

    data = data.rename_axis(['date', 'ticker'])
    new_order = ['Open', 'High', 'Low','Close','Volume', 'Adj Close']
    new_names = ['open', 'high', 'low','close','volume', 'Adj Close']

    # Reordenar las columnas
    prices = data[new_order]

    # Renombrar las columnas
    prices.columns = new_names
    prices = prices.sort_index()
    prices = prices.loc[~prices.index.duplicated(keep='first')]

    prices = prices.swaplevel(0, 1)
    prices = prices.sort_index(level=list(range(len(prices.index.names))))

    


    return prices

def save_data(prices):
    with pd.HDFStore(DATA_STORE) as store:
        store.put('data_close', prices.sort_index().reset_index())
        print(store.info())



In [3]:
ticker_list = ['XLE', 'XLB', 'XLI', 'XLK', 'XLF', 
               'XLP', 'XLY', 'XLV', 'XLU', 'IYR', 'VOX', 'SPY']

data = download_data(ticker_list)
prices = process_data(data)

save_data(prices)
prices

[*********************100%***********************]  12 of 12 completed


<class 'pandas.io.pytables.HDFStore'>
File path: ./data/assets.h5
/data_close            frame        (shape->[77810,8])


open        high         low  \
ticker date                                                            
IYR    2000-06-19 00:00:00+00:00   35.031250   35.031250   35.031250   
       2000-06-20 00:00:00+00:00   35.125000   35.179688   35.125000   
       2000-06-21 00:00:00+00:00   35.390625   35.390625   35.351562   
       2000-06-22 00:00:00+00:00   35.367188   35.367188   35.367188   
       2000-06-23 00:00:00+00:00   35.367188   35.367188   35.367188   
...                                      ...         ...         ...   
XLY    2024-11-01 00:00:00+00:00  200.509995  202.149994  199.899994   
       2024-11-04 00:00:00+00:00  199.250000  201.050003  198.910004   
       2024-11-05 00:00:00+00:00  199.850006  202.990005  199.679993   
       2024-11-06 00:00:00+00:00  207.550003  209.589996  205.899994   
       2024-11-07 00:00:00+00:00  209.830002  212.960007  209.759995   

                                       close     volume   Adj Close  
ticker date                                                          
IYR    2000-06-19 00:00:00+00:00   35.031250      200.0   13.283124  
       2000-06-20 00:00:00+00:00   35.179688      400.0   13.339415  
       2000-06-21 00:00:00+00:00   35.351562    20600.0   13.404583  
       2000-06-22 00:00:00+00:00   35.367188      400.0   13.410504  
       2000-06-23 00:00:00+00:00   35.367188        0.0   13.410504  
...                                      ...        ...         ...  
XLY    2024-11-01 00:00:00+00:00  200.089996  3803800.0  200.089996  
       2024-11-04 00:00:00+00:00  199.399994  2384600.0  199.399994  
       2024-11-05 00:00:00+00:00  202.929993  2212100.0  202.929993  
       2024-11-06 00:00:00+00:00  209.419998  4699300.0  209.419998  
       2024-11-07 00:00:00+00:00  212.110001  4010500.0  212.110001  

[77810 rows x 6 columns]

In [4]:
data

Ticker                            VOX                                      \
Price                            Open        High         Low       Close   
Date                                                                        
1993-01-29 00:00:00+00:00         NaN         NaN         NaN         NaN   
1993-02-01 00:00:00+00:00         NaN         NaN         NaN         NaN   
1993-02-02 00:00:00+00:00         NaN         NaN         NaN         NaN   
1993-02-03 00:00:00+00:00         NaN         NaN         NaN         NaN   
1993-02-04 00:00:00+00:00         NaN         NaN         NaN         NaN   
...                               ...         ...         ...         ...   
2024-11-01 00:00:00+00:00  148.800003  149.690002  148.570007  148.630005   
2024-11-04 00:00:00+00:00  148.539993  148.759995  147.289993  147.699997   
2024-11-05 00:00:00+00:00  148.279999  149.619995  148.279999  149.399994   
2024-11-06 00:00:00+00:00  151.360001  153.369995  151.220001  153.169998   
2024-11-07 00:00:00+00:00  153.850006  155.639999  153.720001  155.289993   

Ticker                                                  XLI              \
Price                       Adj Close    Volume        Open        High   
Date                                                                      
1993-01-29 00:00:00+00:00         NaN       NaN         NaN         NaN   
1993-02-01 00:00:00+00:00         NaN       NaN         NaN         NaN   
1993-02-02 00:00:00+00:00         NaN       NaN         NaN         NaN   
1993-02-03 00:00:00+00:00         NaN       NaN         NaN         NaN   
1993-02-04 00:00:00+00:00         NaN       NaN         NaN         NaN   
...                               ...       ...         ...         ...   
2024-11-01 00:00:00+00:00  148.630005  121100.0  134.509995  135.240005   
2024-11-04 00:00:00+00:00  147.699997   74900.0  134.149994  134.880005   
2024-11-05 00:00:00+00:00  149.399994   61700.0  134.330002  136.130005   
2024-11-06 00:00:00+00:00  153.169998  185300.0  140.669998  141.679993   
2024-11-07 00:00:00+00:00  155.289993   84600.0  141.830002  141.889999   

Ticker                                             ...        XLE             \
Price                             Low       Close  ...        Low      Close   
Date                                               ...                         
1993-01-29 00:00:00+00:00         NaN         NaN  ...        NaN        NaN   
1993-02-01 00:00:00+00:00         NaN         NaN  ...        NaN        NaN   
1993-02-02 00:00:00+00:00         NaN         NaN  ...        NaN        NaN   
1993-02-03 00:00:00+00:00         NaN         NaN  ...        NaN        NaN   
1993-02-04 00:00:00+00:00         NaN         NaN  ...        NaN        NaN   
...                               ...         ...  ...        ...        ...   
2024-11-01 00:00:00+00:00  133.979996  134.009995  ...  87.849998  88.029999   
2024-11-04 00:00:00+00:00  133.490005  133.860001  ...  88.510002  89.570000   
2024-11-05 00:00:00+00:00  133.990005  136.100006  ...  89.610001  90.169998   
2024-11-06 00:00:00+00:00  139.169998  141.429993  ...  92.050003  93.559998   
2024-11-07 00:00:00+00:00  140.389999  140.580002  ...  92.419998  93.129997   

Ticker                                                  XLF             \
Price                      Adj Close      Volume       Open       High   
Date                                                                     
1993-01-29 00:00:00+00:00        NaN         NaN        NaN        NaN   
1993-02-01 00:00:00+00:00        NaN         NaN        NaN        NaN   
1993-02-02 00:00:00+00:00        NaN         NaN        NaN        NaN   
1993-02-03 00:00:00+00:00        NaN         NaN        NaN        NaN   
1993-02-04 00:00:00+00:00        NaN         NaN        NaN        NaN   
...                              ...         ...        ...        ...   
2024-11-01 00:00:00+00:00  88.029999  13152300.0  46.630001  47.000000   
2024-11-0

## Obtenemos los precios del fin de semana

In [5]:
prices = prices.loc[(slice(None), slice(str(START), str(END))), "close"].unstack('ticker')
             

In [6]:
prices

ticker,IYR,SPY,VOX,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
date,,,,,,,,,,,,
2005-01-03 00:00:00+00:00,60.834999,120.300003,54.790001,29.430000,34.980000,24.711617,30.799999,20.930000,23.080000,27.530001,29.840000,35.060001
2005-01-04 00:00:00+00:00,60.000000,118.830002,54.580002,28.900000,34.799999,24.476036,30.420000,20.540001,23.000000,27.340000,29.600000,34.650002
2005-01-05 00:00:00+00:00,57.755001,118.010002,53.900002,28.500000,34.650002,24.435417,30.209999,20.469999,22.879999,26.910000,29.549999,34.520000
2005-01-06 00:00:00+00:00,58.250000,118.610001,53.959999,28.690001,35.250000,24.557270,30.309999,20.410000,22.990000,27.049999,29.770000,34.340000
2005-01-07 00:00:00+00:00,58.244999,118.440002,54.250000,28.740000,34.970001,24.427296,30.170000,20.450001,23.100000,27.049999,29.750000,34.330002
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-01 00:00:00+00:00,97.199997,571.039978,148.630005,93.129997,88.029999,46.639999,134.009995,223.910004,80.150002,78.129997,147.740005,200.089996
2024-11-04 00:00:00+00:00,98.250000,569.809998,147.699997,93.540001,89.570000,46.279999,133.860001,223.740005,80.370003,77.199997,146.770004,199.399994
2024-11-05 00:00:00+00:00,99.589996,576.700012,149.399994,93.680000,90.169998,46.709999,136.100006,226.889999,80.879997,78.360001,147.850006,202.929993


In [7]:
tiene_indices_duplicados = prices.index.duplicated().any()
tiene_indices_duplicados


np.False_

In [8]:
weekly_prices = prices.resample('W').last()
weekly_prices.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1036 entries, 2005-01-09 00:00:00+00:00 to 2024-11-10 00:00:00+00:00
Freq: W-SUN
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   IYR     1036 non-null   float64
 1   SPY     1036 non-null   float64
 2   VOX     1036 non-null   float64
 3   XLB     1036 non-null   float64
 4   XLE     1036 non-null   float64
 5   XLF     1036 non-null   float64
 6   XLI     1036 non-null   float64
 7   XLK     1036 non-null   float64
 8   XLP     1036 non-null   float64
 9   XLU     1036 non-null   float64
 10  XLV     1036 non-null   float64
 11  XLY     1036 non-null   float64
dtypes: float64(12)
memory usage: 105.2 KB


In [9]:
outlier_cutoff = 0.01
data = pd.DataFrame()
#lags = [1, 2, 3, 6,9, 12]
lags = [1, 2, 3, 6, 12, 52] #para semanas
for lag in lags:
    data[f'return_{lag}w'] = (weekly_prices
                           .pct_change(lag)
                           .stack()
                           .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                  upper=x.quantile(1-outlier_cutoff)))
                           .add(1)
                           .pow(1/lag)
                           .sub(1)
                           )
data = data.swaplevel().dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11808 entries, ('IYR', Timestamp('2006-01-08 00:00:00+0000', tz='UTC')) to ('XLY', Timestamp('2024-11-10 00:00:00+0000', tz='UTC'))
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   return_1w   11808 non-null  float64
 1   return_2w   11808 non-null  float64
 2   return_3w   11808 non-null  float64
 3   return_6w   11808 non-null  float64
 4   return_12w  11808 non-null  float64
 5   return_52w  11808 non-null  float64
dtypes: float64(6)
memory usage: 629.0+ KB


In [10]:
data.loc['SPY']

,return_1w,return_2w,return_3w,return_6w,return_12w,return_52w
date,,,,,,
2006-01-08 00:00:00+00:00,0.031564,0.006605,0.005457,0.001710,0.006615,0.001560
2006-01-15 00:00:00+00:00,0.001869,0.016608,0.005024,0.002390,0.007154,0.001628
2006-01-22 00:00:00+00:00,-0.021060,-0.009662,0.003893,-0.000476,0.004194,0.001458
2006-01-29 00:00:00+00:00,0.020402,-0.000544,0.000259,0.002855,0.004286,0.001740
2006-02-05 00:00:00+00:00,-0.017660,0.001190,-0.006282,-0.000645,0.001675,0.000943
...,...,...,...,...,...,...
2024-10-13 00:00:00+00:00,0.011519,0.007071,0.006602,0.004647,0.004529,0.005690
2024-10-20 00:00:00+00:00,0.008644,0.010080,0.007595,0.013199,0.005947,0.006324
2024-10-27 00:00:00+00:00,-0.009494,-0.000466,0.003513,0.004988,0.006944,0.006629


In [11]:
min_obs = 52*10 #semanal
nobs = data.groupby(level='ticker').size()
keep = nobs[nobs>min_obs].index

data = data.loc[pd.IndexSlice[keep,:], :]
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11808 entries, ('IYR', Timestamp('2006-01-08 00:00:00+0000', tz='UTC')) to ('XLY', Timestamp('2024-11-10 00:00:00+0000', tz='UTC'))
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   return_1w   11808 non-null  float64
 1   return_2w   11808 non-null  float64
 2   return_3w   11808 non-null  float64
 3   return_6w   11808 non-null  float64
 4   return_12w  11808 non-null  float64
 5   return_52w  11808 non-null  float64
dtypes: float64(6)
memory usage: 629.0+ KB


In [12]:
data.describe()

,return_1w,return_2w,return_3w,return_6w,return_12w,return_52w
count,11808.000000,11808.000000,11808.000000,11808.000000,11808.000000,11808.000000
mean,0.001748,0.001586,0.001496,0.001387,0.001319,0.001218
std,0.026496,0.018077,0.014750,0.010236,0.007150,0.003611
min,-0.081870,-0.058888,-0.050530,-0.037055,-0.026945,-0.011653
25%,-0.012061,-0.007437,-0.005619,-0.003538,-0.001778,-0.000242
50%,0.002895,0.002902,0.002826,0.002435,0.002233,0.001725
75%,0.016684,0.012154,0.010076,0.007544,0.005650,0.003359
max,0.078132,0.048714,0.039536,0.026384,0.016940,0.009113


In [13]:
data2 = data.copy()

In [14]:
data.head()

return_1w  return_2w  return_3w  return_6w  \
ticker date                                                                    
IYR    2006-01-08 00:00:00+00:00   0.048324   0.014821   0.006070   0.002022   
       2006-01-15 00:00:00+00:00  -0.001487   0.023116   0.009356   0.002630   
       2006-01-22 00:00:00+00:00  -0.003425  -0.002457   0.014191   0.003198   
       2006-01-29 00:00:00+00:00   0.029886   0.013094   0.008210   0.007140   
       2006-02-05 00:00:00+00:00  -0.013349   0.008037   0.004202   0.006775   

                                  return_12w  return_52w  
ticker date                                               
IYR    2006-01-08 00:00:00+00:00    0.008728    0.002768  
       2006-01-15 00:00:00+00:00    0.008687    0.002748  
       2006-01-22 00:00:00+00:00    0.006316    0.002515  
       2006-01-29 00:00:00+00:00    0.009771    0.003757  
       2006-02-05 00:00:00+00:00    0.005039    0.002898

## Normalizado de retornos

In [15]:
def normalize_by_rolling_std(series):
    return series / series.rolling(52).std().shift(1)

def normalize_returns(data):
    lags = [1, 2, 3, 6, 12, 52] #para semanas
    for lag in lags:
           data[f'return_{lag}w'] = (data.groupby(level='ticker')[f'return_{lag}w']
                             .transform(normalize_by_rolling_std)) # Esto lo he cambiado para que funcione, debería estar bien
                             
    return data

def neutralize(group):
    return (group - group.mean()) / group.std()

def neutralize_returns(data):
    lags = [1, 2, 3, 6, 12, 52] #para semanas
    for lag in lags:
           data[f'return_{lag}w'] = (data.groupby(level='date')[f'return_{lag}w']
                             .transform(neutralize))
           
    return data


normaliza = True
neutraliza = True


if normaliza:
    data = normalize_returns(data)

if neutraliza:
    data = neutralize_returns(data)



In [28]:
data

return_1w  return_2w  return_3w  return_6w  \
ticker date                                                                    
IYR    2006-01-08 00:00:00+00:00        NaN        NaN        NaN        NaN   
       2006-01-15 00:00:00+00:00        NaN        NaN        NaN        NaN   
       2006-01-22 00:00:00+00:00        NaN        NaN        NaN        NaN   
       2006-01-29 00:00:00+00:00        NaN        NaN        NaN        NaN   
       2006-02-05 00:00:00+00:00        NaN        NaN        NaN        NaN   
...                                     ...        ...        ...        ...   
XLY    2024-10-13 00:00:00+00:00  -0.919311  -1.017466  -0.498940   0.927325   
       2024-10-20 00:00:00+00:00   0.139124  -0.673528  -1.280409   0.263489   
       2024-10-27 00:00:00+00:00   1.760660   1.477718   0.925216   0.599331   
       2024-11-03 00:00:00+00:00   0.749696   1.809587   1.808385   0.902372   
       2024-11-10 00:00:00+00:00   0.938859   0.918274   1.487945   0.636921   

                                  return_12w  return_52w  
ticker date                                               
IYR    2006-01-08 00:00:00+00:00         NaN         NaN  
       2006-01-15 00:00:00+00:00         NaN         NaN  
       2006-01-22 00:00:00+00:00         NaN         NaN  
       2006-01-29 00:00:00+00:00         NaN         NaN  
       2006-02-05 00:00:00+00:00         NaN         NaN  
...                                      ...         ...  
XLY    2024-10-13 00:00:00+00:00   -0.963910   -0.554463  
       2024-10-20 00:00:00+00:00    0.966606   -0.131547  
       2024-10-27 00:00:00+00:00    2.801730    0.028157  
       2024-11-03 00:00:00+00:00    2.951194   -0.042644  
       2024-11-10 00:00:00+00:00    2.296164    0.120584  

[11808 rows x 6 columns]

In [16]:

data2

return_1w  return_2w  return_3w  return_6w  \
ticker date                                                                    
IYR    2006-01-08 00:00:00+00:00   0.048324   0.014821   0.006070   0.002022   
       2006-01-15 00:00:00+00:00  -0.001487   0.023116   0.009356   0.002630   
       2006-01-22 00:00:00+00:00  -0.003425  -0.002457   0.014191   0.003198   
       2006-01-29 00:00:00+00:00   0.029886   0.013094   0.008210   0.007140   
       2006-02-05 00:00:00+00:00  -0.013349   0.008037   0.004202   0.006775   
...                                     ...        ...        ...        ...   
XLY    2024-10-13 00:00:00+00:00  -0.008023  -0.010662  -0.000186   0.008254   
       2024-10-20 00:00:00+00:00   0.014905   0.003375  -0.002212   0.015056   
       2024-10-27 00:00:00+00:00   0.006516   0.010702   0.004421   0.007086   
       2024-11-03 00:00:00+00:00  -0.003585   0.001453   0.005917   0.002860   
       2024-11-10 00:00:00+00:00   0.060073   0.027751   0.020623   0.009141   

                                  return_12w  return_52w  
ticker date                                               
IYR    2006-01-08 00:00:00+00:00    0.008728    0.002768  
       2006-01-15 00:00:00+00:00    0.008687    0.002748  
       2006-01-22 00:00:00+00:00    0.006316    0.002515  
       2006-01-29 00:00:00+00:00    0.009771    0.003757  
       2006-02-05 00:00:00+00:00    0.005039    0.002898  
...                                      ...         ...  
XLY    2024-10-13 00:00:00+00:00    0.003810    0.004092  
       2024-10-20 00:00:00+00:00    0.007403    0.005284  
       2024-10-27 00:00:00+00:00    0.010974    0.005679  
       2024-11-03 00:00:00+00:00    0.011546    0.004283  
       2024-11-10 00:00:00+00:00    0.012318    0.005268  

[11808 rows x 6 columns]

In [26]:
neutralize_returns(data).groupby(level='ticker').agg(['mean', 'std'])

return_1w           return_2w           return_3w           return_6w  \
            mean       std      mean       std      mean       std      mean   
ticker                                                                         
IYR    -0.061188  1.061031 -0.094017  1.043746 -0.117433  1.037508 -0.137659   
SPY     0.049184  0.466425  0.074307  0.454944  0.087820  0.458756  0.109810   
VOX    -0.046331  1.037155 -0.049996  1.030762 -0.061888  1.008960 -0.075807   
XLB    -0.015830  0.868155 -0.022024  0.846153 -0.032812  0.821727 -0.044636   
XLE    -0.072736  1.190207 -0.135699  1.189626 -0.149563  1.211696 -0.181062   
XLF    -0.067875  0.884867 -0.081823  0.887682 -0.097673  0.902601 -0.136507   
XLI     0.005413  0.708708  0.022188  0.714430  0.026831  0.710521  0.025816   
XLK     0.094918  0.917728  0.139737  0.905507  0.175972  0.905692  0.221075   
XLP     0.036767  1.017156  0.042240  1.009975  0.049188  1.018304  0.046399   
XLU    -0.008821  1.262513 -0.011739  1.276526 -0.029579  1.264294 -0.033061   
XLV     0.045397  0.993198  0.055370  1.017290  0.077366  1.016755  0.113865   
XLY     0.041101  0.804562  0.061455  0.804790  0.071770  0.806204  0.091766   

                 return_12w           return_52w            
             std       mean       std       mean       std  
ticker                                                      
IYR     0.980491  -0.169971  0.969407  -0.486076  0.826419  
SPY     0.477437   0.154698  0.505162   0.274942  0.481375  
VOX     1.017449  -0.111105  0.968214  -0.119466  0.953469  
XLB     0.794688  -0.073062  0.758072  -0.136668  0.687982  
XLE     1.243561  -0.281476  1.252229  -0.484675  1.354446  
XLF     0.893573  -0.207076  0.904768  -0.545533  0.901307  
XLI     0.703625  -0.005767  0.693541   0.077752  0.617104  
XLK     0.908559   0.323099  0.910941   0.578006  0.888246  
XLP     0.982417   0.085603  0.936760   0.348130  0.864383  
XLU     1.289257  -0.030083  1.282326  -0.137845  1.028878  
XLV     1.008392   0.190756  1.025466   0.343752  0.885053  
XLY     0.825587   0.124384  0.822372   0.287681  0.865271

In [27]:
normalize_returns(data).groupby(level='ticker').agg(['mean', 'std'])

return_1w           return_2w           return_3w           return_6w  \
            mean       std      mean       std      mean       std      mean   
ticker                                                                         
IYR    -0.047745  1.012298 -0.076615  1.029870 -0.100789  1.033272 -0.133995   
SPY     0.117559  1.084333  0.183936  1.091510  0.223395  1.092912  0.306909   
VOX    -0.038436  1.022225 -0.038562  1.036779 -0.047704  1.050599 -0.067011   
XLB    -0.032863  1.013671 -0.046653  1.022190 -0.065961  1.033510 -0.073187   
XLE    -0.080840  1.025598 -0.147690  1.046175 -0.159905  1.068265 -0.194503   
XLF    -0.041270  1.018601 -0.050396  1.031779 -0.058411  1.042643 -0.095523   
XLI    -0.006658  1.019366  0.014929  1.018470  0.015754  1.030078  0.014349   
XLK     0.112775  1.031478  0.164003  1.038946  0.205057  1.052813  0.277449   
XLP     0.026554  1.018189  0.029167  1.021737  0.030919  1.031629  0.019179   
XLU    -0.015372  1.006237 -0.022970  1.014186 -0.037680  1.025162 -0.041579   
XLV     0.051149  1.026340  0.067581  1.037503  0.099143  1.047168  0.167793   
XLY     0.088863  1.024354  0.135484  1.039708  0.162986  1.052574  0.209760   

                 return_12w           return_52w            
             std       mean       std       mean       std  
ticker                                                      
IYR     1.059191  -0.218741  1.197468  -0.990064  1.519524  
SPY     1.185991   0.501406  1.448538   1.266007  2.403694  
VOX     1.103024  -0.116593  1.255999  -0.633454  2.357457  
XLB     1.072273  -0.126573  1.182281  -0.651155  1.977387  
XLE     1.172114  -0.351240  1.320747  -1.524377  2.905141  
XLF     1.070685  -0.173515  1.140215  -1.124023  1.758628  
XLI     1.058788  -0.043372  1.157454  -0.049812  2.039551  
XLK     1.100616   0.491275  1.229500   1.584906  2.160869  
XLP     1.066000   0.073967  1.130096   0.902496  2.175368  
XLU     1.055772  -0.060078  1.167677  -0.217856  1.460892  
XLV     1.112420   0.301476  1.213659   1.239239  2.244783  
XLY     1.093051   0.302408  1.187226   0.778872  1.948303

In [17]:
data

return_1w  return_2w  return_3w  return_6w  \
ticker date                                                                    
IYR    2006-01-08 00:00:00+00:00        NaN        NaN        NaN        NaN   
       2006-01-15 00:00:00+00:00        NaN        NaN        NaN        NaN   
       2006-01-22 00:00:00+00:00        NaN        NaN        NaN        NaN   
       2006-01-29 00:00:00+00:00        NaN        NaN        NaN        NaN   
       2006-02-05 00:00:00+00:00        NaN        NaN        NaN        NaN   
...                                     ...        ...        ...        ...   
XLY    2024-10-13 00:00:00+00:00  -0.829516  -0.885473  -0.411691   0.688868   
       2024-10-20 00:00:00+00:00   0.124560  -0.587841  -1.056769   0.198762   
       2024-10-27 00:00:00+00:00   1.558272   1.261160   0.745645   0.449819   
       2024-11-03 00:00:00+00:00   0.681582   1.580819   1.465140   0.671557   
       2024-11-10 00:00:00+00:00   0.858654   0.826534   1.249818   0.480204   

                                  return_12w  return_52w  
ticker date                                               
IYR    2006-01-08 00:00:00+00:00         NaN         NaN  
       2006-01-15 00:00:00+00:00         NaN         NaN  
       2006-01-22 00:00:00+00:00         NaN         NaN  
       2006-01-29 00:00:00+00:00         NaN         NaN  
       2006-02-05 00:00:00+00:00         NaN         NaN  
...                                      ...         ...  
XLY    2024-10-13 00:00:00+00:00   -0.500673   -0.365445  
       2024-10-20 00:00:00+00:00    0.494795   -0.086834  
       2024-10-27 00:00:00+00:00    1.463889    0.018568  
       2024-11-03 00:00:00+00:00    1.707089   -0.027799  
       2024-11-10 00:00:00+00:00    1.471550    0.077635  

[11808 rows x 6 columns]

In [18]:
# Calcular nulos por ticker
nulos_por_ticker = data.groupby(level='ticker').apply(lambda x: x.isnull().sum())
print("\nNulos por ticker:")
print(nulos_por_ticker)



Nulos por ticker:
        return_1w  return_2w  return_3w  return_6w  return_12w  return_52w
ticker                                                                    
IYR            52         52         52         52          52          52
SPY            52         52         52         52          52          52
VOX            52         52         52         52          52          52
XLB            52         52         52         52          52          52
XLE            52         52         52         52          52          52
XLF            52         52         52         52          52          52
XLI            52         52         52         52          52          52
XLK            52         52         52         52          52          52
XLP            52         52         52         52          52          52
XLU            52         52         52         52          52          52
XLV            52         52         52         52          52          52
XLY   

In [19]:
# Calcular no nulos por ticker
no_nulos_por_ticker = data.groupby(level='ticker').apply(lambda x: x.notna().sum())
print("\nNo nulos por ticker:")
print(no_nulos_por_ticker)



No nulos por ticker:
        return_1w  return_2w  return_3w  return_6w  return_12w  return_52w
ticker                                                                    
IYR           932        932        932        932         932         932
SPY           932        932        932        932         932         932
VOX           932        932        932        932         932         932
XLB           932        932        932        932         932         932
XLE           932        932        932        932         932         932
XLF           932        932        932        932         932         932
XLI           932        932        932        932         932         932
XLK           932        932        932        932         932         932
XLP           932        932        932        932         932         932
XLU           932        932        932        932         932         932
XLV           932        932        932        932         932         932
XLY

In [20]:
data.index.get_level_values(0).unique()

Index(['IYR', 'SPY', 'VOX', 'XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU',
       'XLV', 'XLY'],
      dtype='object', name='ticker')

In [21]:
data.loc['SPY']

,return_1w,return_2w,return_3w,return_6w,return_12w,return_52w
date,,,,,,
2006-01-08 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-15 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-22 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-29 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2006-02-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-10-13 00:00:00+00:00,0.560194,0.388904,0.548215,0.485868,0.199707,1.422656
2024-10-20 00:00:00+00:00,-0.034128,0.403636,0.478775,0.816112,0.435614,1.369868
2024-10-27 00:00:00+00:00,0.432636,0.333536,0.859415,0.545134,0.820186,1.317652


In [22]:
df_SPY = data.loc['SPY']
new_df = pd.DataFrame()
for ticker in data.index.get_level_values(0).unique():
    if ticker != 'SPY':
        df_temp = data.loc[ticker] - df_SPY
        df_temp['ticker'] = ticker
        new_df = pd.concat([new_df, df_temp])

new_df.set_index('ticker', append=True, inplace=True)
new_df = new_df.reorder_levels(['ticker', 'date'])

In [23]:
df_SPY

,return_1w,return_2w,return_3w,return_6w,return_12w,return_52w
date,,,,,,
2006-01-08 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-15 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-22 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-29 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2006-02-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-10-13 00:00:00+00:00,0.560194,0.388904,0.548215,0.485868,0.199707,1.422656
2024-10-20 00:00:00+00:00,-0.034128,0.403636,0.478775,0.816112,0.435614,1.369868
2024-10-27 00:00:00+00:00,0.432636,0.333536,0.859415,0.545134,0.820186,1.317652
